In [ ]:
def extract_sequences_from_fasta(fasta_file, genes):
    sequences = []
    
    fasta_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    for gene in genes:
        chrom,start, end, gene_name = gene
        gene_seq = fasta_dict['chr6'].seq[start-1:end]
        record = (gene_name, str(gene_seq).upper())
        sequences.append(record)

    return sequences

In [ ]:
all_gene_sequences = extract_sequences_from_fasta(fasta_file, all_genes_in_region)
pc_gene_sequences = extract_sequences_from_fasta(fasta_file, pc_genes_in_region)

In [ ]:
def write_gene_fastas(fasta_dir, attr, gene_sequences):
    Path(fasta_dir).mkdir(parents=True, exist_ok=True)
    with open(f'{fasta_dir}/{region}_{attr}.fasta', 'w+') as f:
        for gene, sequence in gene_sequences:
            f.write(f'>{gene}\n{sequence}\n')
    return f'{fasta_dir}/{region}_{attr}.fasta'

In [ ]:
pc_fasta_file = write_gene_fastas(f'{workdir}/gene_fastas', 'protein_coding', pc_gene_sequences)
all_fasta_file = write_gene_fastas(f'{workdir}/gene_fastas', 'all', all_gene_sequences)

In [ ]:
alignments_dir = f'{workdir}/gene_alignments'
pc_alignmnet_file = f'{alignments_dir}/aligned_pc_{region}.gaf'
all_alignmnet_file = f'{alignments_dir}/aligned_all_{region}.gaf'
Path(alignments_dir).mkdir(exist_ok=True, parents=True)

In [ ]:
align_pc_genes = f'GraphAligner -g {viz_output} -f {pc_fasta_file} -t 32  -a {pc_alignmnet_file} -x vg --multimap-score-fraction 0.1'
align_all_genes= f'GraphAligner -g {viz_output} -f {all_fasta_file}  -t 32 -a {all_alignmnet_file} -x vg --multimap-score-fraction 0.1'

In [ ]:
align_pc_genes

'GraphAligner -g chr15#73077637_73177637.arp-v2.wd/chr15#73077637_73177637.1.arp-v2.walks.gfa -f chr15#73077637_73177637.arp-v2.wd/gene_fastas/chr15#73077637_73177637_protein_coding.fasta -t 32  -a chr15#73077637_73177637.arp-v2.wd/gene_alignments/aligned_pc_chr15#73077637_73177637.gaf -x vg --multimap-score-fraction 0.1'

In [ ]:
if gfab.split('.')[0] == 'CPC':
    os.system(f'sed -i "/^L/ s/\*/0M/g" {connected_output}')
    

In [ ]:
os.system(f"awk '($1 == \"S\" && $3 != \"*\") ' {connected_output} > tmp_file && mv tmp_file {connected_output}" )

0

In [ ]:
align_all_genes

'GraphAligner -g chr15#73077637_73177637.arp-v2.wd/chr15#73077637_73177637.1.arp-v2.walks.gfa -f chr15#73077637_73177637.arp-v2.wd/gene_fastas/chr15#73077637_73177637_all.fasta  -t 32 -a chr15#73077637_73177637.arp-v2.wd/gene_alignments/aligned_all_chr15#73077637_73177637.gaf -x vg --multimap-score-fraction 0.1'

In [ ]:
# with ProcessPoolExecutor() as executor:
#     executor.map(run_command, [align_pc_genes, align_all_genes])
os.system(align_all_genes)
os.system(align_pc_genes)

GraphAligner bioconda 1.0.17-


GraphAligner bioconda 1.0.17-
Load graph from chr15#73077637_73177637.arp-v2.wd/chr15#73077637_73177637.2.arp-v2.gfa
Build alignment graph
Build minimizer seeder from the graph
Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Extend up to 5 seed clusters
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
Backtrace from 10 highest scoring local maxima per cluster
write alignments to chr15#73077637_73177637.arp-v2.wd/gene_alignments/aligned_all_chr15#73077637_73177637.gaf
Align
Alignment finished
Input reads: 10 (53006bp)
Seeds found: 872
Seeds extended: 22
Reads with a seed: 5 (45862bp)
Reads with an alignment: 5 (3102bp)
Alignments: 265 (48821bp) (15 additional alignments discarded)
End-to-end alignments: 0 (0bp)
GraphAligner bioconda 1.0.17-
Load graph from chr15#73077637_73177637.arp-v2.wd/chr15#73077637_73177637.2.arp-v2.gfa
Build alignment graph
Build minimizer seeder from the graph


GraphAligner bioconda 1.0.17-


Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Extend up to 5 seed clusters
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
Backtrace from 10 highest scoring local maxima per cluster
write alignments to chr15#73077637_73177637.arp-v2.wd/gene_alignments/aligned_pc_chr15#73077637_73177637.gaf
Align
Alignment finished
Input reads: 2 (34525bp)
Seeds found: 674
Seeds extended: 10
Reads with a seed: 2 (34525bp)
Reads with an alignment: 2 (2354bp)
Alignments: 206 (37954bp) (12 additional alignments discarded)
End-to-end alignments: 0 (0bp)


0

In [ ]:
def get_segement_colors_from_alignment(alignment_file, color_files_dir):
    df_all = pd.read_csv(alignment_file, sep='\t', header=None)
    df_all['index'] = df_all.groupby(0).cumcount() + 1
    df = df_all[[0, 5]]
    df = df.groupby(0).agg(lambda x: ''.join(x)).reset_index()
    def random_color():
        return "#%06x" % random.randint(0, 0xFFFFFF)

    df['color'] = df.apply(lambda x: random_color(), axis=1)


    items=[]
    for i, row in df.iterrows():
        gene = str(row[0])
        nodes = re.split(r'[<>]',row[5])
        with open(f'{color_files_dir}/{gene}.csv', 'w') as f:
            
        
            for node in nodes:
                if node != '':
                    f.write(f'{node}, {row["color"]}\n')
                    items.append((node, row['color']))
    return pd.DataFrame(items), df_all

In [ ]:
color_files_dir = f'{workdir}/color_csvs'
Path(color_files_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
pc_gene_colors, pc_genes_all = get_segement_colors_from_alignment(pc_alignmnet_file, color_files_dir)
pc_gene_colors.to_csv(f'{color_files_dir}/pc_colors.csv', header=None, index=None)
all_gene_colors, all_genes_all = get_segement_colors_from_alignment(all_alignmnet_file, color_files_dir)
all_gene_colors.to_csv(f'{color_files_dir}/all_colors.csv', header=None, index=None)